In [1]:
import sys
sys.path.append("../")
import transformers
transformers.logging.set_verbosity_error()

from functools import partial
from transformers import AutoTokenizer, AutoModelForMaskedLM
from src.data.dataio import DataFiles, Dataset, remove_empty_fn, truncate_fn
from src.models.tokenizing import Vocabulary, Tokenizer

In [2]:
data_files = DataFiles.from_url_file(url_file="../data/books.txt")

dataset = Dataset(data_files)
dataset = dataset.map(remove_empty_fn, batched=True)
vocab = Vocabulary(dataset["text"])
tokenizer = Tokenizer(vocab=vocab)
dataset = dataset.map(
    partial(truncate_fn, tokenizer=tokenizer, max_seq_length=24, fill_to_max=True), batched=True)

Using custom data configuration default-36d446ce666f48eb
Reusing dataset text (/home/pavlo/.cache/huggingface/datasets/text/default-36d446ce666f48eb/0.0.0/e10c27cd9493b3e5d594862e2a0d24e29034e10b895b72057984ed60b5a1c333)
Loading cached processed dataset at /home/pavlo/.cache/huggingface/datasets/text/default-36d446ce666f48eb/0.0.0/e10c27cd9493b3e5d594862e2a0d24e29034e10b895b72057984ed60b5a1c333/cache-3f285d44524c65e6.arrow


  0%|          | 0/50 [00:00<?, ?ba/s]

AttributeError: 'Tokenizer' object has no attribute 'sep_token'

In [ ]:

from src.models.rnn_model import RNNMaskedLM, RNNLanguageModel


tokenizer = Tokenizer(vocab=vocab)
rnn = RNNLanguageModel(tokenizer=tokenizer, num_layers=1, bidirectional=False)


In [ ]:
from torch import nn, optim


loss_fn = nn.CrossEntropyLoss(ignore_index=vocab.pad_token_id)
_optim = optim.Adam(rnn.parameters(), lr=0.001)

rnn.fit(dataset=dataset, loss_fn=loss_fn, optim=_optim, n_epochs=2, batch_size=2)